In [2]:
import torch as t
import numpy as np
import preprocssing
from torch.utils.data import DataLoader, Dataset

c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [175]:
class TweetsDataset(Dataset):
    def __init__(self, csvPath):
        '''initialize the data and labels'''
        super().__init__()
        data, labels = preprocssing.preprocessDF(csvPath)
        self.data = data
        self.labels = t.tensor(list(map(lambda x: 2 if x == -1 else x, t.tensor(labels)[:, 1])))
        self.data_size = len(data)
        self.vocabs = self.calculate_vocabs()
        self.vocab_size = len(self.vocabs)
        self.data_padding()
        self.replace_with_vocab_id()
        self.data = t.tensor(self.data)
    
    def __getitem__(self, index):
        ''' get the row item'''
        return self.data[index], self.labels[index]

    def data_padding(self):
        max_length = self.get_max_data_length()
        for data_row in self.data:
            if len(data_row) < max_length:
                padding = max_length - len(data_row)
                padding_list = [' '] * padding
                data_row += padding_list

    def get_max_data_length(self):
        max_length = -1e9
        for data_row in self.data:
            if len(data_row) > max_length:
                max_length = len(data_row)
        return max_length

    def calculate_vocabs(self):
        '''Calculate the vocabs book from the data'''
        vocabs_set = set()
        vocab_book = {}
        for tweet in self.data:
            vocabs_set.update(tweet)
        # here I started with token 1 as to leave zero token to the unknown word
        for i, word in enumerate(vocabs_set):
            vocab_book[word] = i + 1
        vocab_book['UNK'] = 0
        return vocab_book

    def replace_with_vocab_id(self):
        '''Replace the string word with its id in vocabs book'''
        for i, sentence in enumerate(self.data):
            self.data[i] = [self.vocabs[word] if word in self.vocabs else self.vocabs['UNK'] for word in sentence]


    def __len__(self):
        '''get the dataset size'''
        return self.data_size

In [184]:
class RNN(t.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_dim
        self.embedding = t.nn.Embedding(vocab_size, embedding_dim=embedding_dim)
        self.rnn = t.nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = t.nn.Linear(hidden_size, num_classes)

    def forward(self, input):
        input = self.embedding(t.tensor(input))
        out, _ = self.rnn(input)
        # out = out[:, -1, :]
        out = self.fc(out)
        return out


In [185]:
class TweetsClassifier():
    def __init__(self):
        pass
    def train(self):
        pass
    def predict(self):
        pass

In [186]:
class RnnTweetsClassifier(TweetsClassifier):
    def __init__(self, dataset, embedding_dim, hidden_size, num_layers, num_classes, epoch_size, learning_rate):
        super().__init__()
        self.dataset = dataset
        self.embedding_dim = embedding_dim
        self.epoch_size = epoch_size
        self.learning_rate = learning_rate
        self.model = RNN(dataset.vocab_size, embedding_dim, hidden_size, num_layers, num_classes)
        self.data_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=True)
        self.criterion = t.nn.CrossEntropyLoss()
        self.optimizer = t.optim.Adam(self.model.parameters(), self.learning_rate)
         
    def train(self):
        for epoch in range(self.epoch_size):
            total_model_acc = 0
            for data in self.data_loader:
                train_data = data[:][0]
                labels = data[:][1]
                # reshape the row data
                output = self.model(train_data)
                # calculate the accuracy
                # print(output.shape)
                total_model_acc += (t.argmax(output[:, -1, :], dim=1) == labels).sum()
                # print(output[:, -1, :].shape)
                loss = self.criterion(output[:, -1, :], labels)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            total_model_acc = total_model_acc / len(self.dataset)
            print(f"epoch num: {epoch} has accuracy: {total_model_acc}")

    def predict(self, testLoader):
        with t.no_grad():
            n_correct = 0
            n_samples = 0
            for tweets in testLoader:
                test_data = tweets[:][0]
                labels = tweets[:][1]
                output = self.model(test_data)
                n_correct += (t.argmax(output[:, -1, :], dim=1) == labels).sum()
                n_samples += labels.size(0)
            print(f"Model accuracy: {100 * n_correct / n_samples}")


In [187]:
# load the dataset and dataloader
dataset = TweetsDataset('./dataset/train.csv')
tweet_classifier = RnnTweetsClassifier(dataset, 50, 50, 1, 3, 10, 0.001)

In [188]:
tweet_classifier.train()

C:\Users\omara\AppData\Local\Temp\ipykernel_26388\4013883744.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))


epoch num: 0 has accuracy: 0.6270749568939209
epoch num: 1 has accuracy: 0.7925014495849609
epoch num: 2 has accuracy: 0.7925014495849609
epoch num: 3 has accuracy: 0.7925014495849609
epoch num: 4 has accuracy: 0.7925014495849609
epoch num: 5 has accuracy: 0.7925014495849609
epoch num: 6 has accuracy: 0.7925014495849609
epoch num: 7 has accuracy: 0.7925014495849609
epoch num: 8 has accuracy: 0.7925014495849609
epoch num: 9 has accuracy: 0.7925014495849609


In [189]:
testset = TweetsDataset('./dataset/dev.csv')
testloader = DataLoader(dataset=testset, batch_size=256, shuffle=True)

In [190]:
tweet_classifier.predict(testloader)

C:\Users\omara\AppData\Local\Temp\ipykernel_26388\4013883744.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = self.embedding(t.tensor(input))


Model accuracy: 80.4000015258789
